In [1]:
import shap
import numpy as np
import keras
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.05
from keras.backend.tensorflow_backend import set_session
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import CuDNNLSTM
from keras.layers import concatenate
from keras.layers import Reshape
from keras import optimizers

In [3]:
rnn_input = Input(shape=(14*19,), name='rnn_input')
reshaped = Reshape((14,19))(rnn_input)
right = CuDNNLSTM(64)(reshaped)

x = Dense(64, activation='relu')(right)
x = Dropout(0.5)(x)
x = Dense(32, activation='relu')(x)
output = Dense(1, activation='sigmoid', name='output')(x)

model = Model(inputs=[rnn_input], outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [4]:
x_train_1 = np.random.randint(0, 100, size=(100,14*19))
x_test_1 = np.random.randint(0, 100, size=(10,14*19))

# Skipping the simple_model.fit(...) for now

explainer = shap.DeepExplainer(model, [x_train_1])
shap_values = explainer.shap_values([x_train_1])

In [9]:
y_base = explainer.expected_value[0]
y_pred = model.predict(x_train_1)
print 'y_base',y_base
print 'y_pred_mean',y_pred.mean()

y_base 0.39998502
y_pred_mean 0.39998502


### check the diff between the shap output & model output

In [27]:
y_diff = abs(y_pred.reshape(100,) - (y_base + shap_values[0][:].sum(axis=1)))

In [28]:
print 'max:',y_diff.max()
print 'min:',y_diff.min()
print 'mean:',y_diff.mean()

max: 0.14786111109132227
min: 0.0018124761780183296
mean: 0.05757006030661598


### question
- Why the sum of all features + y_base not equal to the model output ? 
- Can the deepexplain deal with the mulit input like (,14,19) instead of (,14*19) ?
- hope to get the answer thanks